In [1]:
# !wget https://github.com/alexeygrigorev/minsearch/blob/main/minsearch.py

# !uv pip install minsearch

In [1]:
from minsearch import Index
import json

from groq import Groq
import os

In [2]:
with open("documents.json", "rt") as f_in:
    docs_raw = json.load(f_in)

# docs_raw

In [4]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict["documents"]:
        doc["course"] = course_dict["course"]
        documents.append(doc)

documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [5]:
# Create and fit the index
index = Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [7]:
client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

In [8]:
def search(query):
    boost = {"question": 3.0, "section": 0.5}
    
    results = index.search(
        query=query,
        filter_dict={"course": "data-engineering-zoomcamp"},
        boost_dict=boost,
        num_results=5
    )

    return results

In [9]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [10]:
def llm(prompt):

    response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="llama-3.3-70b-versatile",
)
    return response.choices[0].message.content.strip()

In [11]:
query  = "how do i run kafka"

def rag(query):
    search_results = search(query)

    prompt = build_prompt(query, search_results)

    answer = llm(prompt)
    return answer

In [12]:
rag("hello there")

'Hello, how can I assist you with the course today? Do you have a specific question about one of the modules or workshops?'

In [13]:
from elasticsearch import Elasticsearch

In [14]:
es_client = Elasticsearch("http://localhost:9200")

es_client.info()

ObjectApiResponse({'name': '74eb47c8cdd5', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'TeC1iWH_RZO26uKRpvjS6w', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

index in elasetic is like table in a realational database

In [17]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

# es_client.indices.create(index=index_name, body=index_settings)

In [18]:
for doc in documents:
    es_client.index(index=index_name, document=doc)

In [19]:

query = 'I just disovered the course. Can I still join it?'

In [20]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [21]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [22]:
rag(query)

"Yes, you can still join the course after the start date. You're still eligible to submit the homeworks, but be aware that there will be deadlines for turning in the final projects, so don't leave everything for the last minute."